In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.chdir("/content/drive/My Drive/ml_proj")

In [5]:
%%shell
pip install scanpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 KB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 88.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 KB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.0/296.0 KB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 KB 9.9 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=714271c6537d524e3154e7a1625664a936dc86f923f0

In [6]:
import os
import numpy as np
import scanpy as sc
from copy import deepcopy
import umap.umap_ as umap
import warnings
from torch import nn, optim
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader
from sklearn.metrics import adjusted_mutual_info_score, normalized_mutual_info_score
warnings.filterwarnings('ignore')

In [36]:
import torch
import numpy as np
from torch import nn


def create_conv_block(in_channels, out_channels, kernel_size, stride):
    return nn.Sequential(
        nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride),
        nn.BatchNorm1d(out_channels),
        nn.GELU(),
    )


class CNN(nn.Module):
    def __init__(self, k):
        super().__init__()
        stride = 1 if k == 500 else 2 if k == 1000 else 3
        self.conv_block1 = create_conv_block(4, 64, 1, stride)  # shape: [n, 64, 500]
        self.conv_block2 = create_conv_block(64, 128, 7, 4)  # shape: [n, 128, 124]
        self.conv_block3 = create_conv_block(128, 256, 5, 4)  # shape: [n, 256, 30]
        self.conv_block4 = create_conv_block(256, 512, 5, 4)  # shape: [n, 512, 7]
        self.linear_block = nn.Sequential(
            nn.Linear(512 * 7, 64),
            nn.Dropout(.2),
            nn.GELU(),
            nn.Linear(64, 2000),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = x.reshape(-1, 512 * 7)
        x = self.linear_block(x)
        return x


class AlternativeCNN(nn.Module):
    def __init__(self, k):
        super().__init__()
        stride = 1 if k == 500 else 2 if k == 1000 else 3
        self.conv_block1 = nn.Sequential(
            nn.Conv1d(4, 64, (1, ), (stride, )),  # shape: [n, 64, 500]
            nn.BatchNorm1d(64),
            nn.GELU(),
            nn.MaxPool1d(2, 2),  # shape: [n, 64, 250]
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv1d(64, 256, (3, ), (4, )),  # shape: [n, 256, 62]
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.MaxPool1d(2, 2),  # shape: [n, 256, 31]
        )
        self.conv_block3 = nn.Sequential(
            nn.Conv1d(256, 512, (5, ), (2, )),  # shape: [n, 512, 14]
            nn.BatchNorm1d(512),
            nn.GELU(),
            nn.MaxPool1d(2, 2)  # shape: [n, 512, 7]
        )
        self.linear_block = nn.Sequential(
            nn.Linear(512 * 7, 64),
            nn.BatchNorm1d(64),
            nn.Dropout(.2),
            nn.GELU(),
            nn.Linear(64, 2000),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = x.reshape(-1, 512 * 7)
        x = self.linear_block(x)
        return x


class AlternativeCNN1(nn.Module):
    def __init__(self, k):
        super().__init__()
        stride = 1 if k == 500 else 2 if k == 1000 else 3
        self.conv_block1 = nn.Sequential(
            nn.Conv1d(4, 64, (1, ), (stride, )),  # shape: [n, 64, 500]
            nn.BatchNorm1d(64),
            nn.GELU(),
            nn.MaxPool1d(2, 2),  # shape: [n, 64, 250]
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv1d(64, 256, (3, ), (4, )),  # shape: [n, 256, 62]
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.MaxPool1d(2, 2),  # shape: [n, 256, 31]
        )
        self.conv_block3 = nn.Sequential(
            nn.Conv1d(256, 64, (5, ), (4, )),  # shape: [n, 64, 7]
            nn.BatchNorm1d(64),
            nn.GELU(),
            nn.MaxPool1d(7)  # shape: [n, 64, 1]
        )
        self.linear_block = nn.Sequential(
            nn.Linear(64, 2000),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = x.reshape(-1, 64)
        x = self.linear_block(x)
        return x


class AlternativeCNN2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(4, 64, kernel_size=(9, ), stride=(3, ))  # shape: [n, 64, 164]
        self.b = nn.BatchNorm1d(64)
        self.g = nn.GELU()
        self.p = nn.MaxPool1d(2, 2)  # shape: [n, 64, 82]
        self.conv_block1 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=(3, ), stride=(2, )),  # shape: [n, 128, 40]
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.MaxPool1d(2, 2)  # shape: [n, 128, 20]
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv1d(128, 256, kernel_size=(5, ), stride=(2, )),  # shape: [n, 256, 8]
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.MaxPool1d(2, 2)  # shape: [n, 256, 4]
        )
        self.linear_block = nn.Sequential(
            nn.Linear(256 * 4, 64),
            nn.BatchNorm1d(64),
            nn.Dropout(.2),
            nn.Linear(64, 2000),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.p(self.g(self.b(self.conv1(x))))
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = x.reshape(-1, 256 * 4)
        x = self.linear_block(x)
        return x

    def get_feature_map(self, x):
        return self.conv1(x)


def train(net, loader, cri, opt, device, max_epoch):
    net.train()
    loss_list = []
    for epoch in range(max_epoch):
        cur_loss = .0
        for idx, (data, label) in enumerate(loader):
            data = data.to(device)
            label = label.to(device)
            output = net(data)
            loss = cri(output, label)
            opt.zero_grad()
            loss.backward()
            opt.step()
            cur_loss += loss.item()
        cur_loss /= len(loader)
        loss_list.append(cur_loss)
        print('epoch: {}, loss: {:.6f}'.format(epoch+1, cur_loss))
    return loss_list


def test(net, loader, device):
    net.eval()
    res = []
    for idx, (data, label) in enumerate(loader):
        data = data.to(device)
        output = net(data)
        output = output.detach().cpu().numpy()
        res.append(output)
    res = np.concatenate(res, axis=0)
    return res


def get_conv_map(net, loader, device):
    net.eval()
    f_map_list = []
    for (data, label) in loader:
        net = net.eval()
        data = data.to(device)
        f_map = net.get_feature_map(data).detach().cpu().numpy()
        f_map_list.append(f_map)
    f_map_list = np.concatenate(f_map_list)
    return f_map_list


In [37]:
class AlternativeCNN3_1000(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(4, 32, kernel_size=(9,), stride=(2, ))  # shape: [n, 32, 496]
        self.b = nn.BatchNorm1d(32)
        self.g = nn.GELU()
        self.p = nn.MaxPool1d(3, 3)  # shape: [n, 32, 165]
        self.conv_block1 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size=(6,), stride=(1,)),  # shape: [n, 64, 160]
            nn.BatchNorm1d(64),
            nn.GELU(),
            nn.MaxPool1d(2, 2)  # shape: [n, 64, 80]
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=(7,), stride=(1,)),  # shape: [n, 128, 64]
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.MaxPool1d(2, 2)  # shape: [n, 128, 32]
        )
        self.conv_block3 = nn.Sequential(
            nn.Conv1d(128, 256, kernel_size=(5,), stride=(1,)),  # shape: [n, 256, 28]
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.MaxPool1d(2, 2)  # shape: [n, 256, 14]
        )
        self.conv_block4 = nn.Sequential(
            nn.Conv1d(256, 512, kernel_size=(5,), stride=(1,)),  # shape: [n, 512, 10]
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.MaxPool1d(2, 2)  # shape: [n, 256, 5]
        )
        self.conv_block4 = nn.Sequential(
            nn.Conv1d(256, 512, kernel_size=(5,), stride=(1,)),  # shape: [n, 512, 10]
            nn.BatchNorm1d(512),
            nn.GELU(),
            nn.MaxPool1d(2, 2)  # shape: [n, 256, 5]
        )
        self.conv_block5 = nn.Sequential(
            nn.Conv1d(512, 1024, kernel_size=(3,), stride=(1,)),  # shape: [n, 512, 3]
            nn.BatchNorm1d(1024),
            nn.GELU(),
            nn.MaxPool1d(3, 3)  # shape: [n, 512, 1]
        )
        self.linear_block = nn.Sequential(
            nn.Linear(1024, 64),
            nn.BatchNorm1d(64),
            nn.Dropout(.2),
            nn.Linear(64, 2000, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.p(self.g(self.b(self.conv1(x))))
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.conv_block5(x)
        x = x.reshape(-1, 1024)
        x = self.linear_block(x)
        return x
    
    def get_feature_map(self, x):
        return self.conv1(x)

In [9]:
import torch
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    def __init__(self, data, label):
        super().__init__()
        self.data = data
        self.label = label

    def __getitem__(self, item):
        data = torch.FloatTensor(self.data[item].reshape(4, -1))
        label = torch.FloatTensor(self.label[item])
        return data, label

    def __len__(self):
        return len(self.data)


In [33]:
import numpy as np
from copy import deepcopy
from sklearn.metrics import roc_auc_score


def transfer_letter_to_num(data):
    res = []
    res_int = []
    for seq in data:
        tmp = deepcopy(seq)
        tmp = tmp.replace('A', '0')
        tmp = tmp.replace('G', '1')
        tmp = tmp.replace('C', '2')
        tmp = tmp.replace('T', '3')
        tmp = np.array([int(s) for s in list(tmp)[:-1]])
        res_int.append(tmp.reshape(1, -1))
        one_hot = np.zeros(shape=(len(tmp), 4))
        one_hot[np.arange(len(tmp)), tmp] = 1
        res.append(one_hot.reshape((1, -1, 4)))
    res = np.concatenate(res)
    res_int = np.concatenate(res_int)
    return res, res_int


def clean_str(data):
    res = []
    for seq in data:
        seq = seq.replace('\n', '')
        res.append(seq)
    return res


def cal_auc(pred, label):
    res = .0
    res_list = []
    for idx in range(len(label)):
        tmp = roc_auc_score(label[idx], pred[idx])
        res += tmp
        res_list.append(tmp)
    res /= len(label)
    return res_list, res

In [11]:
def get_motif(f_map, ratio, data_str, data_int, data_mat, k_size, stride):
    res_str, res_int = [], []
    res_mat = np.zeros(shape=(k_size, 4))
    for chan in range(f_map.shape[1]):
        f_map_chan = f_map[chan]
        thr = ratio * np.max(f_map_chan)
        index = np.where(f_map_chan > thr)
        batch_index, seq_index = index
        seq_index = seq_index * stride
        for idx, batch in enumerate(batch_index):
            seq_idx = seq_index[idx]
            res_str.append(data_str[batch][seq_idx: seq_idx+k_size])
            res_int.append(data_int[batch][seq_idx: seq_idx+k_size])
            res_mat += data_mat[batch][seq_idx: seq_idx+k_size]
    return res_str, res_int, res_mat.transpose(1, 0)

In [34]:
path_train = '/data/train'
path_test = './data/test'
path_cell_type = './data/celltype.txt'
train_path_list = []
test_path_list = []
for idx in ['train', 'test']:
    for item in [500, 1000, 1500]:
        if idx == 'train':
            train_path_list.append('data/train/{}/sequences_train.txt'.format(item))
        else:
            test_path_list.append('data/test/{}/sequences_test.txt'.format(item))

with open(train_path_list[1], 'r') as f:
    train_1000 = f.readlines()

with open(test_path_list[1], 'r') as f:
    test_1000 = f.readlines()

with open(path_cell_type, 'r') as f:
    cell = f.readlines()


f_mat_train_1000 = sc.read('./data/train/1000/matrix_train.mtx')
label_train_1000 = np.array(f_mat_train_1000.X.todense()).astype(np.int_)

f_mat_test_1000 = sc.read('./data/test/1000/matrix_test.mtx')
label_test_1000 = np.array(f_mat_test_1000.X.todense()).astype(np.int_)

mat_train_1000, int_train_1000 = transfer_letter_to_num(train_1000)
mat_test_1000, int_test_1000 = transfer_letter_to_num(test_1000)
str_test_1000 = clean_str(test_1000)

In [38]:
max_epoch = 40
batch_size = 256
lr = 1e-3
wd = 5e-4
device = 'cuda'

train_dataset = CustomDataset(mat_train_1000, label_train_1000)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = CustomDataset(mat_test_1000, label_test_1000)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
net = AlternativeCNN3_1000().to(device)
cri = nn.BCELoss().to(device)
opt = optim.Adam(net.parameters(), lr=lr, weight_decay=wd)

loss_list = train(net, train_loader, cri, opt, device, max_epoch)

epoch: 1, loss: 0.633172
epoch: 2, loss: 0.381876
epoch: 3, loss: 0.324602
epoch: 4, loss: 0.306070
epoch: 5, loss: 0.296613
epoch: 6, loss: 0.290881
epoch: 7, loss: 0.286966
epoch: 8, loss: 0.284400
epoch: 9, loss: 0.282566
epoch: 10, loss: 0.281043
epoch: 11, loss: 0.280101
epoch: 12, loss: 0.279281
epoch: 13, loss: 0.278757
epoch: 14, loss: 0.278357
epoch: 15, loss: 0.277685
epoch: 16, loss: 0.277182
epoch: 17, loss: 0.276729
epoch: 18, loss: 0.276434
epoch: 19, loss: 0.276133
epoch: 20, loss: 0.275912
epoch: 21, loss: 0.275880
epoch: 22, loss: 0.275637
epoch: 23, loss: 0.275373
epoch: 24, loss: 0.275377
epoch: 25, loss: 0.275208
epoch: 26, loss: 0.275114
epoch: 27, loss: 0.275093
epoch: 28, loss: 0.275029
epoch: 29, loss: 0.275138
epoch: 30, loss: 0.274847
epoch: 31, loss: 0.274788
epoch: 32, loss: 0.274898
epoch: 33, loss: 0.274892
epoch: 34, loss: 0.274847
epoch: 35, loss: 0.274674
epoch: 36, loss: 0.274727
epoch: 37, loss: 0.274811
epoch: 38, loss: 0.274734
epoch: 39, loss: 0.27

In [39]:
pred = test(net, test_loader, device)
auc_list, auc = cal_auc(pred, label_test_1000)

In [40]:
auc

0.7047699195618626

In [41]:
net

AlternativeCNN3_1000(
  (conv1): Conv1d(4, 32, kernel_size=(9,), stride=(2,))
  (b): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (g): GELU(approximate='none')
  (p): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv_block1): Sequential(
    (0): Conv1d(32, 64, kernel_size=(6,), stride=(1,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU(approximate='none')
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block2): Sequential(
    (0): Conv1d(64, 128, kernel_size=(7,), stride=(1,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU(approximate='none')
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block3): Sequential(
    (0): Conv1d(128, 256, kernel_size=(5,), stride=(1,))
    (1): BatchNorm1d(256, eps=1e-05, moment

In [42]:
auc_array = np.array(auc_list)
np.save('./auc_array_1000.npy', auc_array)

In [43]:
f_map = get_conv_map(net, test_loader, device)

In [44]:
res_str, res_int, res_mat = get_motif(f_map, .98, str_test_1000, int_test_1000, mat_test_1000, 9, 1)

In [45]:
res_mat

array([[ 3.,  8.,  1.,  7.,  4., 10.,  6.,  3.,  4.],
       [14., 15., 21., 15., 15., 10., 16., 19., 15.],
       [18., 13., 14., 15., 16., 12., 11., 14., 17.],
       [ 5.,  4.,  4.,  3.,  5.,  8.,  7.,  4.,  4.]])

In [46]:
res_mat_ratio = res_mat / np.sum(res_mat, axis=0)
print(res_mat_ratio)

[[0.075 0.2   0.025 0.175 0.1   0.25  0.15  0.075 0.1  ]
 [0.35  0.375 0.525 0.375 0.375 0.25  0.4   0.475 0.375]
 [0.45  0.325 0.35  0.375 0.4   0.3   0.275 0.35  0.425]
 [0.125 0.1   0.1   0.075 0.125 0.2   0.175 0.1   0.1  ]]


In [47]:
res_str_set = set(res_str)
for idx, item in enumerate(res_str_set):
    if idx % 4 == 3:
        print(item)
    else:
        print(item, end=' ')

GAGGGATCC CAGCTCCGC GGGGCGGGC CAGCGGTCC
CACCTGTTT CCCGCCCGC CGCCAAGTC GGCAGTGTG
CCGCGCCCG CCCCCTCGC CCCGTCTGC GTGAGGGCA
GCCCGTGGG CGGGCTCCG GGCGGCGGA CCCGCTCCT
CCTCCTGCC CCGGAGAGC TCCTGCTCC AGGAGGGCT
CGGCGGGCG GTGAGGGCG GAGCCAGAC AAGACACCC
GGGCTCCGC TGCGTGAAG CCGGCCAGG GATCCAGGA
CCGCGCGGC CTCGCTCAG GGATGAACA GGCTCCGCG
TCTGCACGC TGGGGCGGG AGGAATCGG TTGACATTG
CGCCCCAGG GCGGGATGG GAGCCAAGT CGTGAGGGC


In [48]:
clus_data = net.linear_block[3].weight.cpu().detach().numpy()

In [49]:
cell_str = ''.join(cell)
cell_str = cell_str.replace('\n', '')
cell_str = cell_str.replace('CLP', '0')
cell_str = cell_str.replace('CMP', '1')
cell_str = cell_str.replace('GMP', '2')
cell_str = cell_str.replace('HSC', '3')
cell_str = cell_str.replace('LMPP', '4')
cell_str = cell_str.replace('MEP', '5')
cell_str = cell_str.replace('MPP', '6')
cell_str = cell_str.replace('pDC', '7')
cell_str = cell_str.replace('UNK', '8')
cell_str = cell_str.replace('mono', '9')
cell_int = np.array([int(s) for s in cell_str])


clus_k = KMeans(n_clusters=10)
clus_res = clus_k.fit_predict(clus_data)

tsne = TSNE(n_components=2, perplexity=30, learning_rate='auto')
dim_rdc_res = tsne.fit_transform(clus_data)


ami = adjusted_mutual_info_score(cell_int, clus_res)
nmi = normalized_mutual_info_score(cell_int, clus_res)

In [50]:
print(ami)
print(nmi)

0.03868807722272804
0.04834913981953502


In [52]:
np.save('./clus_data_1000.npy', clus_data)